# Document Q&A with RAG

Two Big limitations of LLMs:

1. that they only "know" the information that they were trained on, and
2. that they have limited input context windows.

A way to address both of these limitations is to use a technique called Retrival Augmented Generation, or RAG. RAG system has three stages:

1. Indexing
2. Retrival
3. Generaion

**Indexing** happens ahead of time, and allows you to quickly look up relevant information at query-time. When a query comes in, you **retrieve** relevant documents, combine them with your instructions and the user's query, and have the LLM **generate** a tailored answer in natural language using the supplied information.

This allows you to provide information that the model hasn't seen before, such as product-specific knowledge or live weather updates.

<div class="alert alert-block alert-info">
<b>Info:</b> use the Gemini API to create a vector database, retrieve answers to questions from the database and generate a final answer. You will use Chroma, an open-source vector database. With Chroma, you can store embeddings alongside metadata, embed documents and queries, and search your documents.
</div>



In [20]:

import google.generativeai as genai
from IPython.display import Markdown
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])



**EmbedContent API method to calculate embeddings**

In [2]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)


models/embedding-001
models/text-embedding-004


**Data**

In [7]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]
#print(DOCUMENT1, "\n", DOCUMENT2, "\n", DOCUMENT3)


## Creating the embedding database wih chromaDB

- Create a custom function to generate embeddings with the Gemini API.
- Implementing retrival system,
 - `task-type` for generating the document embeddings is `retrieval_document`.
 - `retrieval_query` for the query embeddings.

In [8]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

In [34]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input:Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"


        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model = "models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

Now create a chroma database client that uses the `GeminiEmbeddingFunciton` and populate the database with the documents defined above.

In [35]:
import chromadb

DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2


Confirm that the data was inserted by looking at the database.

In [36]:
db.count()
# You can peek at the data too.
# db.peek(1)

3

## Retrival: Find relevant documents

To search the chroma database, call the `query` method. 

<div class="alert alert-block alert-info">
<b>Note:</b>  Also swich to the `retrieval_query` mode of embedding generation
</div>

In [44]:
# switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chrome DB using the specified query
query = "How do you use the touchscreen to play music"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

## Augmented generation: Answer the question

- Found relevant passage from the set of documents(the retrieval step), Now assemble a generation prompt to have the Gemini API generate a final answer.

<div class="alert alert-block alert-info">
<b>Note:</b>   Note that in this example only a single passage was retrieved. In practice, especially when the size of your underlying data is large, we will want to retrieve more than one result.
</div>

-  let the Gemini model determine what passages are relevant in answering the question.
-  For this reason it's OK if some retrieved passages are not directly related to the question - this generation step should ignore them.

In [45]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""

print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



Now use the `generate_content` method to generate and answer to the question.

In [46]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

You can play music on your Googlecar by simply touching the "Music" icon on the large touchscreen display. 
